In [1]:
'''
ArrNet iplements a TCN architecture to create a model that
refines onsite time estimation on seismic waveform picks.

This notebook is a releasable work in progress that shows how to
instantiate the model and perform a 'dummy' inference with the untrained model.

Note that ArrNet is a fully convolutional network capable of accepting any 
input size.  Care must be taken that the desired features to be excratcted 
fit within the network's receptive field.  The default settings on this 
example have a receptive field of ~30 seconds.
'''

"\nArrNet iplements a TCN architecture to create a model that\nrefines onsite time estimation on seismic waveform picks.\n\nThis notebook is a releasable work in progress that shows how to\ninstantiate the model and perform a 'dummy' inference with the untrained model.\n\nNote that ArrNet is a fully convolutional network capable of accepting any \ninput size.  Care must be taken that the desired features to be excratcted \nfit within the network's receptive field.  The default settings on this \nexample have a receptive field of ~30 seconds.\n"

In [2]:
# If you have downloaded model_utils.py and ArrNet.py to this working directory,
# this line will install them for you.
!pip install -e .

Obtaining file:///home/dickey/PycharmProjects/ArrNet
  Attempting uninstall: sdea
    Found existing installation: sdea 0.0.0
    Uninstalling sdea-0.0.0:
      Successfully uninstalled sdea-0.0.0
  Running setup.py develop for sdea


In [3]:
import os
import numpy as np
import tensorflow as tf

from ArrNet import ArrNet
from model_utils import Params
from model_utils import save_dict_to_json

In [4]:
# Call the get_default_par method to save a set of default parameters in a json file.
# All hyperparameters for the network are controlled by setting values in this par file.

params = ArrNet.get_default_par(os.path.join('.', 'test_par.json'))
params

lr             : 0.001
bs             : 1
loss           : quantile
optimizer      : adam
f_1ow          : 0.8
f_high         : 4.5
f_pad          : 3
s_rate         : 40
w_len          : 3
shift          : 0
f              : 5
d              : [2, 4, 8]
k              : 15
s              : 1
dense          : [3]
pat            : 20
t_step         : 1000
v_step         : 60
cmps           : B
project_name   : 
model_name     : lr:0.001|bs:1|loss:quantile|optimizer:adam|f_1ow:0.8|f_high:4.5|f_pad:3|s_rate:40|w_len:3|shift:0|f:5|d:2x4x8|k:15|s:1|dense:3|pat:20|t_step:1000|v_step:60|cmps:B|trnRET:False|type:tcn|quantiles:0.9772x0.8413x0.1538x0.0228
model_file     : 
model_save     : 
log_folder     : 
data_folder    : 
image_folder   : 
model_folder   : 
trnRET         : False
catalog        : 
type           : tcn
quantiles      : [0.9772, 0.8413, 0.1538, 0.0228]



In [5]:
def get_default_par(par_file): 
    '''
    Parameters
    ------
    par_file : model utils.Param
        A json file where the default parameters will be saved. Only call this if you dont know
        what parameters are needed by the model. You may create your own Param object and
        create the model that way if you know the necessary parameters.
    Returns
    ------
    TYPE
    Default Param object.
    '''

    par = { 'lr': 0.001,
            'bs': 1,
            'loss': 'quantile',
            'optimizer':'adam',
            'f_1ow': 0.8,
            'f_high': 4.5,
            'f_pad': 3 ,
            's_rate': 40,
            'w_len': 3,
            'shift': 0,
            'f': 5,
            'd': [2, 4, 8],
            'k': 15,
            's': 1,
            'dense': [3],
            'pat': 20,
            't_step': 1000,
            'v_step': 60,
            'cmps': 'B',
            'project_name': '', 
            'model_name': '', 
            'model_file': '',
            'model_save': '',
            'log_folder': '',
            'data_folder': '',
            'image_folder': '',
            'model_folder': '',
            'trnRET': False,
            'catalog': '',
            'type': 'tcn',
            'quantiles': [0.9772, 0.8413, 0.1538, 0.0228], }

    save_dict_to_json(par, par_file) 
    return Params(par_file)

def residual_block(x, dilation_rate, nb_filters, kernel_size, padding, dropout_rate=0):
    prev_x = x
    x = tf.keras.layers.Conv1D(filters=nb_filters,
                               kernel_size=kernel_size,
                               dilation_rate=dilation_rate,
                               padding=padding)(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.SpatialDropout1D(rate=dropout_rate)(x)

    # lxl conv to match the shapes (channel dimension)
    prev_x = tf.keras.layers.Conv1D(nb_filters, 1, padding='same')(prev_x) 
    res_x = tf.keras.layers.add([prev_x, x])
    return res_x, x   


def get_loss(q):

    def quantile_loss(q, y_p, y):
        e = y_p-y
        return tf.keras.backend.mean(tf.math.maximum(q*e, (q-1)*e))

    def q_loss(y_true, y_pred):
        return quantile_loss(q, y_true, y_pred) 

    return q_loss

In [6]:
par = params
padding = 'causal'
drop = 0.05
nb_chan = len(par.cmps)
nb_filters = par.f
filter_len = par.k
dilations = par.d
nb_stacks = par.s

In [7]:
input_layer = tf.keras.layers.Input(shape=(None, nb_chan))
x = input_layer
skip_connections = []

In [8]:
for s in range(nb_stacks):
    for d in dilations:
        print(s, d)

0 2
0 4
0 8


In [ ]:


for s in range(nb_stacks):
    for d in dilations:
        x, skip_out = residual_block(x,
                                           dilation_rate=d,
                                           nb_filters=nb_filters,
                                           kernel_size=filter_len,
                                           padding=padding,
                                           dropout_rate=drop)

        skip_connections.append(skip_out)

In [ ]:






x = tf.keras.layers.add(skip_connections)
x = tf.keras.layers.Lambda(lambda tt: tt[:, -1, :])(x)

for dense in par.dense:
    x = tf.keras.layers.Dense(dense, activation='relu')(x)
    if (len(par.dense) > 1):
        x = tf.keras.layers.Dropout(0.2)(x)

if (par.loss == 'quantile'):
    loss_array = {} 
    output_layer = []
    out= tf.keras.layers.Dense(1, activation='linear', name='huber')(x)
    output_layer.append(out)
    loss_array['huber'] = tf.keras.losses.Huber() 
    for q in par.quantiles:
        name= f'output_{int(100*q):02d}'
        out= tf.keras.layers.Dense(1, activation='linear', name=name)(x)
        output_layer.append(out)

        loss_array[name] = get_loss(q)

    model = tf.keras.Model(input_layer, output_layer, name='q_model')
    model.compile(optimizer=par.optimizer, loss=loss_array, metrics=['mean_absolute_error'])
else :
    output_layer = tf.keras.layers.Dense(1, activation='linear', name='output')(x)
    model = tf.keras.Model(input_layer ,output_layer, name='model') 
    model.compile(optimizer=par.optimizer, loss=par.loss, metrics=['mean_absolute_error'])
return model

In [ ]:
# Create a model using the previously created params.
# Settings can be adjusted in the parameters by setting the values
# Ex. params.lr = 0.0001
model = ArrNet.get_network(params)
model.summary()

In [ ]:
# With the model created, lets make a fake data trace to test it
data = np.random.uniform(size=(1,1200,1))

In [ ]:
# The output of this inference corresponds to [0] = y_hat, [1..n] = the chosen percentiles
model(data)